### Coding your own methods.

Sometimes, sk-learn (or other libraries) may not have what you need for your problem, or may need some additional input to give what you need.
In research contexts, you may often want to write your own custom functions for linear regression. 
This notebook walks you through coding some methods "by hand" (with and without assistance from sk-learn). 

We'll be working with the same data we used in class, with 1 feature.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model #New!

font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
#matplotlib.rcParams.update({'figure.autolayout': False})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
np.random.seed(16) #set seed for reproducibility purposes

x = np.arange(100) 

yp = 3*x + 3 + 2*(np.random.poisson(3*x+3,100)-(3*x+3)) #generate some data with scatter following Poisson distribution 
                                                    #with exp value = y from linear model, centered around 0

np.random.seed(12) #set 
out = np.random.choice(100,15) #select 15 outliers indexes
yp_wo = np.copy(yp)
np.random.seed(12) #set again
yp_wo[out] = yp_wo[out] + 5*np.random.rand(15)*yp[out]

plt.scatter(x, yp_wo);

### Custom scores

In studio, we saw that the LinearRegression model allowed us to select from a list of scoring options. We used MSE and MAE.

We might like to implement a scorer where we care about percentage error instead. Here is how to do a custom scorer:

In [ ]:
from sklearn.metrics import make_scorer

Add your own function definition below to return the mean absolute percent error:

$$ MAPE = \frac{100}{n} \Sigma_{i=1}^n |\frac{y_i - \hat{y}_i}{y_i}| $$

where $y_i$'s are the true values and $\hat{y}_i$'s are the predicted values.

In [ ]:
def mape(true,pred): #Modified Mean Absolute Percentage Error
    return # your code here

mape_scorer = make_scorer(mape, greater_is_better = False)

Train a linear regression model using cross-validation. The new $\texttt{mape\_scorer}$ you made can be passed to the "scoring" parameter as usual. 

Give the average test error with uncertainty for the built-in RMSE scorer and your hand-made MAPE scorer.

In [ ]:
# your code to train a LinearRegression model and give RMSE and MAPE

### Gradient Descent Methods

While sk-learn has a built-in stochastic gradient descent module, it doesn't have batch or mini-batch gradient descent (though you can use SGD in a way that approximates those methods). To see how these methods work, we'll code them by hand. That will also let us visualize what they're doing, which we can't do when the steps happen "under the hood" in sk-learn.

We'll do this for MSE, since that allows us to calculate the gradients analytically. For other loss functions, gradients may need to be calculated numerically. 

A reminder from class:

$$ MSE = \frac{1}{N} |X \cdot \beta -y |^2$$

$$ \nabla_\beta MSE = \frac{2}{N} X^T(X \cdot \beta -y )$$

Ultimately, we want to compare the results we get from Gradient Descent methods to the results found using the normal equation. The LinearRegression model you trained above uses the normal equation to solve, so save the coefficients from it to an array for later comparison. 

In [ ]:
# your code to get and save the coefficients for comparison. What happens if you try to do this with cross-validation?

beta_ne = np.array([[intercept],[slope[0]]]) # once you have the coefficients, save them in this form for later computation steps

### Let's now implement the simplest form of gradient descent: batch, stochastic, and mini-batch, one by one.

To begin with, we add x0 = 1 to each instance; this is the bias term and it is used in order to write the solution for a linear model in matrix multiplication form.

In [ ]:
X = np.c_[np.ones((100, 1)), x]  

print(X.shape) #shape is number of instances x number of parameters


We can manually calculate the loss from the result of the normal equation. I'll do this one for you, so it can serve as an example for later steps:

In [ ]:
loss_ne = np.mean((X.dot(beta_ne) - yp_wo.reshape(-1,1))**2)

loss_ne

### Batch GD

For batch gradient descent, you'll use every instance to calculate the gradient at every step, then use the gradient to update the coefficients.

The psuedo-code is in Week 6's lecture! I've provided you a framework where you can fill in the blanks below.

In [ ]:
np.random.seed(10) #same initial conditions for all

eta = 0.0001 # Learning rate, try changing this number!
n_iterations = 1000 #Number of iterations, try changing this number!
m = 100 #number of points

beta_path_bgd = [] #we're going to save the coefficients tested at each iteration in an array, so we can look at the path the gradient descent takes

beta = np.random.randn(2,1) # random initial guesses for the coefficients

for iteration in range(n_iterations):
    gradients = #calculate the gradients for every point. See equation and example above.
    beta = # set the new coefficients. See pseudocode in lecture!
    beta_path_bgd.append(beta) # add the new coefficients to the array

beta_path_bgd = np.array(beta_path_bgd) #save the path

beta_bgd = beta #final result

### Questions:
- Compare the final coefficients and MSE loss from the batch gradient descent method to the results from the normal equation method. 

- What is the percent error on the coefficients from the Batch GD method? In general, does it seem that the slope or intercept is more difficult to determine accurately?

- Try increasing the number of iterations, does the error improve?

In [ ]:
#  a code cell to answer the questions

### Stochastic Gradient Descent

In stochastic gradient descent, a random instance is used to calculate the gradient at each point. Using numpy's $\texttt{random.randint}$ method, pick a random instance at each iteration (you'll need the feature and target values!) and evaluate the gradient for just that instance. 

In [ ]:
np.random.seed(10) #same initial conditions for all

eta = 0.000005 # Learning rate, you'll probably need a smaller one than before.
n_iterations = # Choose a number of iterations, do you think 1000 will be enough?
m = 100 #number of points

beta_path_sgd = [] #we're going to save the coefficients tested at each iteration, so we can look at the path the gradient descent takes

beta = np.random.randn(2,1) # same initial guesses for the coefficients

for iteration in range(n_iterations):
    random_index = #your code to choose a random integer index
    x_one = # get the feature values just for that instance
    y_one = # get the target values just for that instance
    gradients = #calculate the gradient just for that instance. See equation and example above.
    beta = # set the new coefficients. See pseudocode in lecture!
    beta_path_sgd.append(beta) # add the new coefficients to the array

beta_path_sgd = np.array(beta_path_sgd) #save the path

beta_sgd = beta #final result

### Questions: 
- Compare the final coefficients and MSE loss from the stochastic gradient descent method to the results from the normal equation method. 

- What is the percent error on the coefficients from the Stochastic GD method? 

- Try increasing the number of iterations, does the error improve?

- What adjustment to the learning rate can you make to get an improvement? Give the percent error on the coefficients after making that adjustment. Explain why this adjustment helped.

### Mini-batch Gradient Descent

The "in-between" option is Mini-batch Gradient Descent, where you use a small subset of the data (a mini-batch of some size $n$) to calculate the gradient at each iteration. 
You want to use a random mini-batch for each iteration, so a nice way to implement this is to shuffle the list of indices each time, and then choose the first $n$ indices as your mini-batch.

In [ ]:
np.random.seed(10)

beta = np.random.randn(2,1) 

eta = 0.000005

n_iterations = 1000

m = 100 #number of instances

beta_path_mgd = []

minibatch_size = 10 #size of the mini batch

for epoch in range(n_iterations):
    
    shuffled_indices = np.random.permutation(m) #shuffle array 
    
    X_shuffled = X[shuffled_indices]
    
    y_shuffled = yp_wo.reshape(-1,1)[shuffled_indices]
    
    xi = X_shuffled[:minibatch_size] #select the first set from the shuffled array; equivalent to selecting a random subset
    
    yi = y_shuffled[:minibatch_size]
    
    gradients = #gradient for the mini-batch. Make sure you normalize to the right number of points!
    
    beta = #update the coefficients
    
    beta_path_mgd.append(beta)

beta_path_mgd = np.array(beta_path_mgd)

beta_mgd = beta 

print(beta_mgd)

### Questions: 
- Compare the final coefficients and MSE loss from the mini-batch gradient descent method to the results from the normal equation method. 

- What is the percent error on the coefficients from the Mini-Batch GD method? 

- What adjustment to the size of the mini-batch can you make to get an improvement? Give the percent error on the coefficients after making that adjustment. Explain why this adjustment helped.

It's most interesting to actually look at the path taken by GD in the three cases. Increasingly dark colors denote later steps. 

Note: if you changed the number of iterations and didn't change back to the default values, you may run into plotting errors. You can fix them by adjusting the number of color values in parameters like $\texttt{c = np.arange(1000)}$ to match your settings.

In [ ]:
plt.figure(figsize=(14,8))




#plt.text(1.5,3.978,'Normal Equation solution X')
plt.scatter(beta_path_sgd[::10, 0].flatten(), beta_path_sgd[::10, 1].flatten(), marker = 's', s = 5, \
         label="Stochastic GD, N$_{it}$ = 10000", c = np.arange(1000), cmap=plt.cm.Purples)
plt.scatter(beta_path_mgd[:, 0].flatten(), beta_path_mgd[:, 1].flatten(), marker = "+", s = 12, linewidth=1, \
            label="Mini-batch GD, N$_{it}$ = 1000", c = np.arange(1000), cmap=plt.cm.Greens)
plt.scatter(beta_path_bgd[:, 0].flatten(), beta_path_bgd[:, 1].flatten(), marker = "d", s = 12, linewidth=1, \
            label="Batch GD, N$_{it}$ = 1000", c = np.arange(1000,0,-1), cmap=plt.cm.copper)

plt.scatter(beta_sgd[0],beta_sgd[1], marker = "s", s = 100, color = 'Purple', alpha = 0.5)
plt.scatter(beta_mgd[0],beta_mgd[1], marker = "+", s = 200, color = 'DarkGreen', alpha = 1)
plt.scatter(beta_bgd[0],beta_bgd[1], marker = "d", s = 100, color = 'k', alpha = 0.5)
plt.scatter(beta_ne[0],beta_ne[1], marker = "o", s = 100, color = 'r', alpha = 0.5, label="Normal Equation")

legend = plt.legend(loc="upper right", fontsize=16)



for i in range(3):

    legend.legend_handles[i].set_color('k')
    legend.legend_handles[i]._sizes = [30]


plt.xlabel(r"$\beta_0$", fontsize=20)
plt.ylabel(r"$\beta_1$   ", fontsize=20)

plt.axis([1.3, 1.6, 2.5, 6.5])

#plt.savefig('AllThePaths.png', dpi = 300)
plt.show()



One way to improve gradient descent methods is by adding an adaptive learning rate: one that changes during the fit. The simplest case is to simply change the value of the learning rate after some number of iterations. Choose one of the gradient descent methods you coded above, and implement it with a learning rate schedule of your choice. Set it up so that it improves the fit (gets it closer to the normal equation result)!

In [ ]:
# implementation of a gradient descent method with a learning schedule